In [ ]:
import os; HOME_DIR = os.getcwd()
import sys
sys.path.append('/'.join(HOME_DIR.split('/')[:-1]))

In [ ]:
# 모델 불러오기

from detect_faces.ssd import build_deepface_ssd
detector = build_deepface_ssd()

from recognize_faces.deepid import build_deepface_deepid
recognizer = build_deepface_deepid()

In [ ]:
# 초기 얼굴 벡터 DB

from helper.data_manager import load_embeddings_to_df
EMB_DB = load_embeddings_to_df()

# 모자이크 데모 (사진 버젼)

In [ ]:
import hashlib
import ipywidgets as widgets
from IPython.display import display

In [ ]:
# 새로 등록할 사람의 이름을 입력

name_text = widgets.Text(
    placeholder='이름을 영어로 입력하세요',
    description='Name:',
    disabled=False
)

display(name_text)

In [ ]:
# 새로 등록할 사람의 얼굴 사진을 업로드

import cv2

from detect_faces.ssd import detect_faces
from recognize_faces.deepid import preprocess_deepid_input, face_to_embedding
from helper.data_manager import DB_DIR, save_embedding_to_db


file_uploader = widgets.FileUpload(
    accept='image/*',
    multiple=False
)


def upload_image_file(change):
    global EMB_DB
    
    original_file_name = list(file_uploader.value.keys())[0].split('.')
    ext = original_file_name[-1]
    image_content_b = list(file_uploader.value.values())[0]["content"]
    
    hash_obj = hashlib.new('sha224')
    hash_obj.update(image_content_b)
    new_file_name = hash_obj.hexdigest()[:10] # 랜덤하게 업로드 된 사진 파일 이름을 재설정
    
    # 메모리에 업로드 된 사진을 DB 폴더에 저장
    person_name = name_text.value
    if not os.path.exists(os.path.join(DB_DIR, person_name)):
        os.mkdir(os.path.join(DB_DIR, person_name))
    with open(os.path.join(DB_DIR, person_name, new_file_name+'.'+ext), "wb") as fp:
        fp.write(image_content_b)

    # 사진을 불러오고 임베딩 벡터를 생성
    img = cv2.imread(os.path.join(DB_DIR, person_name, new_file_name+'.'+ext))
    faces = detect_faces(detector, img)
    if faces:
        for face in faces:
            detected_face, region_xywh = face
            break # 한개의 얼굴만 DB에 등록
    preprocessed_face = preprocess_deepid_input(detected_face, region_xywh, return_region=False)
    output = face_to_embedding(recognizer, preprocessed_face)

    # 임베딩 벡터를 저장
    save_embedding_to_db(person_name, new_file_name+".pkl", output)
    debug_face = preprocessed_face.squeeze(0) * 255
    cv2.imwrite(os.path.join(DB_DIR, person_name, new_file_name+'_debug.'+ext), debug_face)
    # os.remove(os.path.join(DB_DIR, person_name, new_file_name+'.'+ext)) # 사진 삭제
    
    EMB_DB = load_embeddings_to_df()


file_uploader.observe(upload_image_file, names='_counter')

display(file_uploader)